In [1]:
import pandas as pd
from typing import Set, List, Tuple
import spacy
from spacy.tokens import DocBin
nlp = spacy.load('en_core_web_lg')
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
df = pd.read_json('training_json_file.json', orient='records', encoding='utf-8')
print(df.head())
df['text'] = df['cleaned_html'].replace(r'\n',' ', regex=True)

                                        cleaned_html  label
0                                                      1700
1  1 U.S. 20 1 Dall. 20 1 L.Ed. 19 ANONYMOUS. No....   1700
2  1 U.S. 18 1 Dall. 18 1 L.Ed. 18 The Lessee of ...   1700
3  1 U.S. 194 1 Dall. 194 1 L.Ed. 96 Gerardv.La C...   1700
4  1 U.S. 210 1 Dall. 210 1 L.Ed. 104 Pollardv.Sh...   1700


In [3]:
df_1700s = df.loc[df['label'] == 1700]
df_1800s = df.loc[df['label'] == 1800]
df_1900s = df.loc[df['label'] == 1900]
df_2000s = df.loc[df['label'] == 2000]

In [4]:
df_1800s_sampled = df_1800s.sample(1000)
df_1900s_sampled = df_1900s.sample(1000)
df_2000s_sampled = df_2000s.sample(1000)

In [5]:
combined_df = pd.concat([df_1700s, df_1800s_sampled, df_1900s_sampled, df_2000s_sampled])

In [6]:
combined_df['text'] = combined_df['cleaned_html'].replace(r'\n',' ', regex=True)
combined_df['label'] = combined_df['label'].astype('str')

In [7]:
cats = combined_df.label.unique().tolist()

In [8]:
def make_docs(data, target_file, cats):
    docs = []

    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.append(doc)
    return docs

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(combined_df["text"].values, combined_df["label"].values, test_size=0.2)

In [10]:
tqdm(make_docs(list(zip(X_train, y_train)), "./train_01.spacy", cats=cats))
tqdm(make_docs(list(zip(X_valid, y_valid)), "./valid_01.spacy", cats=cats))
print("Finished making all the docs!")

  0%|          | 0/2621 [01:32<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

from spacy.cli.train import train as spacy_train

config_path = "config.cfg"
output_model_path = "./output"
spacy_train(
    config_path,
    output_path=output_model_path,
    overrides={
        "paths.train": "train_01.spacy",
        "paths.dev": "valid_01.spacy",
    },
)

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================


FileNotFoundError: [Errno 2] No such file or directory: 'train_01.spacy'